In [1]:
import sys
# append the path of the parent directory
sys.path.append("..")

In [64]:
import math
import os
import time


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as patches

import seaborn as sns
import time
import json
import pandas as pd
from ctypes import c_int32
from itertools import product

from scipy.stats import pearsonr
from importlib import reload

from lib import baselines
reload(baselines)

from lib.baselines import CentralDPServer, LDPServer, LDPEncoderGRR, query_df, infer_domains_and_ranges


# set_style()

In [23]:
# Central Differential Privacy (CDP) example:
df = pd.read_parquet("data/clean/online_retail.parquet")
server = CentralDPServer(df, epsilon=10.0)
query = ['*', {'United Kingdom'}, '*', (140, 217), '*', '*']
result = server.query(query, per_query_epsilon=1.0)
print("Noisy Count:", result)
print("Remaining Budget:", server.get_remaining_budget())

Noisy Count: 121542
Remaining Budget: 9.0


In [25]:
# Federated Local Differential Privacy (LDP) example:
df = pd.read_parquet("data/clean/online_retail.parquet")
server = LDPServer(df, epsilon=10.0, number_of_partitions=4)
query = ['*', ['United Kingdom'], '*', (140, 217), '*', '*']
result = server.query(query, per_query_epsilon=0.2)
print("Federated LDP Noisy Count:", result)
print("Remaining Budgets:", server.get_remaining_budgets())

Federated LDP Noisy Count: 121530
Remaining Budgets: [9.8, 9.8, 9.8, 9.8]


In [49]:
# LDP Encoder with GRR example:
df = pd.read_parquet("data/clean/online_retail.parquet")
categorical_domains, numerical_ranges = infer_domains_and_ranges(df)

encoder = LDPEncoderGRR(df=df, epsilon=1.0, categorical_domains=categorical_domains, numerical_ranges=numerical_ranges)
query = ['*', ['United Kingdom'], '*', (140, 217), '*', '*']
result = encoder.query(query)
print("Noisy Count:", result)

Privatizing rows: 100%|██████████| 541909/541909 [00:48<00:00, 11133.96it/s]


Noisy Count: 334


In [55]:
encoder.save_to_file(file_path="baselines/ldp_encoder_grr.json", data_path="baselines/ldp_encoder_grr_data.parquet")

In [65]:
encoder2 = LDPEncoderGRR.load_from_file(file_path="baselines/ldp_encoder_grr.json")